# Test for Cleaner logic

In [2]:
import pandas as pd
from minio_io import download_silver_csv

In [3]:
df = download_silver_csv('2025-10-14-00-26-56')

[cleaner.minio_io] Replaced 'minio' with 'localhost' for host-based testing
[cleaner.minio_io] MinIO config: endpoint=localhost:9000, secure=False, access=admin
[cleaner.minio_io] Attempting to download: s3://transform-data/crash/corr=2025-10-14-00-26-56/merged.csv
[cleaner.minio_io] Found object: size=4374179 bytes, last_modified=2025-10-14 00:27:32+00:00
[cleaner.minio_io] Downloaded Silver CSV: 8683 rows, 32 columns


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8683 entries, 0 to 8682
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   beat_of_occurrence                   8683 non-null   int64  
 1   crash_date                           8683 non-null   object 
 2   crash_day_of_week                    8683 non-null   int64  
 3   crash_hour                           8683 non-null   int64  
 4   crash_record_id                      8683 non-null   object 
 5   crash_type                           8683 non-null   object 
 6   hit_and_run_i                        2432 non-null   object 
 7   injuries_total                       8665 non-null   float64
 8   latitude                             8593 non-null   float64
 9   lighting_condition                   8683 non-null   object 
 10  longitude                            8593 non-null   float64
 11  num_units                     

## Columns to keep (info available at time of report)

'crash_record_id', 'beat_of_occurrence', 'crash_day_of_week', 'crash_hour', 'crash_type', 'hit_and_run_i', 'num_units', 'injuries_total', 'lighting_condition', 'latitude', longitude', 'posted_speed_limit', road_defect', 'roadway_surface_cond', 'street_direction', 'trafficway_type', 'weather_condition', 'intersection_related_i', 'lane_cnt'


In [5]:
print(df['hit_and_run_i'].value_counts())
print(f'Nulls: {sum(df['hit_and_run_i'].isna())}')

hit_and_run_i
Y    2332
N     100
Name: count, dtype: int64
Nulls: 6251


## Step 1: Drop Leakages and ID columns

In [6]:
# Extract columns that we want and make an explicit copy
req_cols = [
    'crash_record_id', 'beat_of_occurrence', 'crash_date', 'crash_day_of_week', 'crash_hour', 'crash_type', 
    'hit_and_run_i', 'num_units', 'injuries_total', 'lighting_condition', 'latitude', 'longitude', 
    'posted_speed_limit', 'road_defect', 'roadway_surface_cond', 'street_direction', 'trafficway_type', 
    'weather_condition', 'intersection_related_i', 'traffic_control_device', 'work_zone_i',
    'private_property_i'
]
existing_cols = []
add_cols = []
df_cols = df.columns

for col in req_cols:
    if col in df_cols:
        existing_cols.append(col)
    else:
        add_cols.append(col)

df_clean = df[existing_cols].copy()
# Init all columns that don't exist to none type
for col in add_cols:
    df_clean[col] = None

df_clean

,crash_record_id,beat_of_occurrence,crash_date,crash_day_of_week,crash_hour,crash_type,hit_and_run_i,num_units,injuries_total,lighting_condition,...,posted_speed_limit,road_defect,roadway_surface_cond,street_direction,trafficway_type,weather_condition,intersection_related_i,traffic_control_device,private_property_i,work_zone_i
0,46c729dc548d987a50b9d9e7de6b019f28932a9deed1ed...,914,2020-01-01T00:00:00.000,4,0,NO INJURY / DRIVE AWAY,Y,2,0.0,"DARKNESS, LIGHTED ROAD",...,30,NO DEFECTS,SNOW OR SLUSH,S,NOT DIVIDED,CLEAR,NaN,NO CONTROLS,NaN,None
1,d2d381e6650d777145ea1fb168b5e0afd8997aff1cd306...,1911,2020-01-01T00:00:00.000,4,0,NO INJURY / DRIVE AWAY,Y,2,0.0,DARKNESS,...,20,NO DEFECTS,DRY,W,NOT DIVIDED,CLEAR,NaN,NO CONTROLS,NaN,None
2,2cbd335f24e9309896f33cdda5f11799c209e564008b0e...,2534,2020-01-01T00:01:00.000,4,0,NO INJURY / DRIVE AWAY,Y,2,0.0,"DARKNESS, LIGHTED ROAD",...,30,NO DEFECTS,SNOW OR SLUSH,N,ONE-WAY,UNKNOWN,NaN,NO CONTROLS,NaN,None
3,7dffbed0feecd57904759d6a8dc858afeec03dd784e2ee...,834,2020-01-01T00:01:00.000,4,0,INJURY AND / OR TOW DUE TO CRASH,N,1,0.0,UNKNOWN,...,25,UNKNOWN,UNKNOWN,S,NOT DIVIDED,UNKNOWN,NaN,NO CONTROLS,NaN,None
4,dc5df961f8f16fe9023d82cdd7b80b949eef13c10e60a0...,1533,2020-01-01T00:10:00.000,4,0,NO INJURY / DRIVE AWAY,NaN,1,0.0,"DARKNESS, LIGHTED ROAD",...,10,NO DEFECTS,ICE,W,ALLEY,CLEAR,NaN,NO CONTROLS,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8678,413cf21225928c24b6cb2147215d8045f37347590755aa...,834,2020-01-31T23:30:00.000,6,23,INJURY AND / OR TOW DUE TO CRASH,NaN,1,0.0,"DARKNESS, LIGHTED ROAD",...,30,NO DEFECTS,SNOW OR SLUSH,W,DIVIDED - W/MEDIAN BARRIER,SNOW,Y,TRAFFIC SIGNAL,NaN,None
8679,82e738f113e57c3f0f677fb250fdf1587a6e034b0fa846...,411,2020-01-31T23:34:00.000,6,23,INJURY AND / OR TOW DUE TO CRASH,NaN,2,0.0,"DARKNESS, LIGHTED ROAD",...,25,NO DEFECTS,WET,E,NOT DIVIDED,SNOW,Y,NO CONTROLS,NaN,None
8680,9001aa47489a7cc35d8dbab6eb23c537a176aace5316f9...,1832,2020-01-31T23:35:00.000,6,23,NO INJURY / DRIVE AWAY,Y,2,0.0,"DARKNESS, LIGHTED ROAD",...,30,NO DEFECTS,WET,N,ONE-WAY,RAIN,Y,TRAFFIC SIGNAL,NaN,None
8681,7a45c3ef1d6b47d8a6127b3f7422f4c212684c9ab65e45...,1931,2020-01-31T23:55:00.000,6,23,NO INJURY / DRIVE AWAY,Y,2,0.0,"DARKNESS, LIGHTED ROAD",...,30,NO DEFECTS,WET,N,DIVIDED - W/MEDIAN (NOT RAISED),SNOW,NaN,NO CONTROLS,NaN,None


## Step 2: Standardize Boolean Columns

In [21]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42144 entries, 0 to 42143
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   crash_record_id         42144 non-null  object 
 1   beat_of_occurrence      42144 non-null  int64  
 2   crash_day_of_week       42144 non-null  int64  
 3   crash_hour              42144 non-null  int64  
 4   crash_type              42144 non-null  object 
 5   hit_and_run_i           15395 non-null  object 
 6   num_units               42144 non-null  int64  
 7   injuries_total          42057 non-null  float64
 8   lighting_condition      42144 non-null  object 
 9   latitude                41858 non-null  float64
 10  longitude               41858 non-null  float64
 11  posted_speed_limit      42144 non-null  int64  
 12  road_defect             42144 non-null  object 
 13  roadway_surface_cond    42144 non-null  object 
 14  street_direction        42144 non-null

In [22]:
# Step 2: Standardize Boolean Columns
# Convert Y/yes/true/1 -> 1, N/no/false/0 -> 0, everything else (including NaN) -> 0

def standardize_boolean(value):
    """
    Standardize boolean-like values to 1 or 0.
    
    Args:
        value: Input value (could be string, int, float, etc.)
    
    Returns:
        1 or 0
    """
    if pd.isna(value):
        return 0
    
    # Convert to string and normalize
    val_str = str(value).strip().lower()
    
    # Map to 1 (True)
    if val_str in ['y', 'yes', 'true', 't', '1', '1.0']:
        return 1
    
    # Everything else (including N/no/false/0 and unknown values) becomes 0
    return 0

# Apply to boolean columns
bool_cols = ['hit_and_run_i', 'intersection_related_i', 'private_property_i', 'work_zone_i']

for col in bool_cols:
    print(f"\n--- Processing {col} ---")
    print(f"Before: {df_clean[col].value_counts(dropna=False)}")
    print(f"Nulls before: {df_clean[col].isna().sum()}")
    
    df_clean[col] = df_clean[col].apply(standardize_boolean)
    
    print(f"\nAfter: {df_clean[col].value_counts(dropna=False)}")
    print(f"Nulls after: {df_clean[col].isna().sum()}")

# Convert to regular integer type (no NaN values now)
for col in bool_cols:
    df_clean[col] = df_clean[col].astype('int64')


--- Processing hit_and_run_i ---
Before: hit_and_run_i
NaN    26749
Y      14775
N        620
Name: count, dtype: int64
Nulls before: 26749

After: hit_and_run_i
0    27369
1    14775
Name: count, dtype: int64
Nulls after: 0

--- Processing intersection_related_i ---
Before: intersection_related_i
NaN    32555
Y       9135
N        454
Name: count, dtype: int64
Nulls before: 32555

After: intersection_related_i
0    33009
1     9135
Name: count, dtype: int64
Nulls after: 0

--- Processing private_property_i ---
Before: private_property_i
NaN    39912
Y       2020
N        212
Name: count, dtype: int64
Nulls before: 39912

After: private_property_i
0    40124
1     2020
Name: count, dtype: int64
Nulls after: 0

--- Processing work_zone_i ---
Before: work_zone_i
NaN    41976
Y        132
N         36
Name: count, dtype: int64
Nulls before: 41976

After: work_zone_i
0    42012
1      132
Name: count, dtype: int64
Nulls after: 0


## Step 3: Clean and create time features
crash_day_of_week, crash_hour

In [23]:
# Create is_weekend column
# Sunday is 1, saturday is 7
def is_weekend_col(val):
    if val == 1 or val == 7:
        return 1
    else:
        return 0
df_clean['is_weekend'] = df_clean['crash_day_of_week'].apply(is_weekend_col)

In [24]:
# Add hour_bin column
def add_hour_bin(val):
    if val <= 6:
        return 'night'
    elif val <= 12:
        return 'morning'
    elif val <= 18:
        return 'afternoon'
    elif val <= 23:
        return 'evening'
    else:
        return None
    
df_clean['hour_bin'] = df_clean['crash_hour'].apply(add_hour_bin)

In [ ]:
# Clean date
# Drop nulls first: nothing to infer date from and cannot fill with median (would make no sense)
df_clean = df_clean.dropna(subset=['crash_date'])

# Convert to date time, normalize to get rid of seconds and time
df_clean['crash_date'] = pd.to_datetime(df_clean['crash_date']).dt.normalize()

## Step 4: Clean Location Features (Latitude/Longitude)

In [25]:
# Chicago approximate bounding box
# Latitude: ~41.64 to ~42.02
# Longitude: ~-87.94 to ~-87.52
CHICAGO_LAT_MIN = 41.6
CHICAGO_LAT_MAX = 42.1
CHICAGO_LNG_MIN = -88.0
CHICAGO_LNG_MAX = -87.5

# Identify invalid rows
invalid_coords = (
    (df_clean['latitude'] == 0) & (df_clean['longitude'] == 0)  # (0, 0) coordinates
) | (
    (df_clean['latitude'] < CHICAGO_LAT_MIN) | (df_clean['latitude'] > CHICAGO_LAT_MAX)  # Outside Chicago lat
) | (
    (df_clean['longitude'] < CHICAGO_LNG_MIN) | (df_clean['longitude'] > CHICAGO_LNG_MAX)  # Outside Chicago lng
)

print(f"\nRows with invalid coordinates: {invalid_coords.sum()}")

# Drop invalid rows
df_clean = df_clean[~invalid_coords].copy()
print(f"Rows after dropping invalid coordinates: {len(df_clean)}")

# Create binned latitude and longitude (rounded to 2 decimal places)
df_clean['lat_bin'] = df_clean['latitude'].round(2)
df_clean['lng_bin'] = df_clean['longitude'].round(2)

# Create grid_id by combining lat_bin and lng_bin
df_clean['grid_id'] = df_clean['lat_bin'].astype(str) + '_' + df_clean['lng_bin'].astype(str)

print(f"\nCreated location bins:")
print(f"  - Unique lat_bin values: {df_clean['lat_bin'].nunique()}")
print(f"  - Unique lng_bin values: {df_clean['lng_bin'].nunique()}")
print(f"  - Unique grid_id values: {df_clean['grid_id'].nunique()}")

print("\nSample grid_ids:")
print(df_clean['grid_id'].head(10))


Rows with invalid coordinates: 1
Rows after dropping invalid coordinates: 42143

Created location bins:
  - Unique lat_bin values: 39
  - Unique lng_bin values: 41
  - Unique grid_id values: 691

Sample grid_ids:
0    41.72_-87.55
1    41.92_-87.77
2      41.9_-87.7
3    41.65_-87.62
4    41.76_-87.56
5    41.75_-87.59
6    41.87_-87.72
7    41.76_-87.63
8    41.87_-87.65
9    41.82_-87.63
Name: grid_id, dtype: object


## Step 5: Road and Environment Features
- roadway_surface_cond
- lighting_condition
- weather_condition
- traffic_control_device
- work_zone_i, work_zone_type
- private_property_i
- crash_type

In [26]:
# Clean roadway_surface_cond
valid_roadway = ['DRY', 'UNKNOWN', 'WET', 'SNOW OR SLUSH', 'ICE']
df_clean['roadway_surface_cond'] = df_clean['roadway_surface_cond'].str.upper()
df_clean.loc[~df_clean['roadway_surface_cond'].isin(valid_roadway), 'roadway_surface_cond'] = 'OTHER'

# Clean lighting_condition
valid_lighting = ['DARKNESS, LIGHTED ROAD', 'UNKNOWN', 'DARKNESS', 'DAWN', 'DAYLIGHT', 'DUSK']
df_clean['lighting_condition'] = df_clean['lighting_condition'].str.upper()
df_clean.loc[~df_clean['lighting_condition'].isin(valid_lighting), 'lighting_condition'] = 'OTHER'

# Consolidate weather_condition
df_clean['weather_condition'] = df_clean['weather_condition'].str.upper()

# Map snow-related conditions to SNOW
snow_conditions = ['SNOW', 'BLOWING SNOW', 'SLEET/HAIL', 'FREEZING RAIN/DRIZZLE']
df_clean.loc[df_clean['weather_condition'].isin(snow_conditions), 'weather_condition'] = 'SNOW'

# Keep common conditions, map others to OTHER
valid_weather = ['CLOUDY/OVERCAST', 'CLEAR', 'RAIN', 'SNOW']
df_clean.loc[~df_clean['weather_condition'].isin(valid_weather), 'weather_condition'] = 'OTHER'

# Clean traffic_control_device
valid_traffic = ['NO CONTROLS', 'TRAFFIC SIGNAL', 'STOP SIGN/FLASHER', 'UNKNOWN']
df_clean['traffic_control_device'] = df_clean['traffic_control_device'].str.upper()
df_clean.loc[~df_clean['traffic_control_device'].isin(valid_traffic), 'traffic_control_device'] = 'OTHER'

# Clean crash_type
valid_crash_type = ['NO INJURY / DRIVE AWAY', 'INJURY AND / OR TOW DUE TO CRASH']
df_clean['crash_type'] = df_clean['crash_type'].str.upper()
df_clean.loc[~df_clean['crash_type'].isin(valid_crash_type), 'crash_type'] = 'OTHER'

## Step 6: Vehicles and People (N/A)

## Step 7: Contributary Cause Grouping (not included for hit and run ML outcome)

## Step 8: Handle Missing Values

- Numeric: Median
- Categorical: Most Frequent OR missing
- Bool: Already handled

In [27]:
# Step 8: Handle Missing Values

# Exception: injuries_total nulls -> 0 (no injuries reported)
df_clean['injuries_total'] = df_clean['injuries_total'].fillna(0)

# Handle numeric columns with median
numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()
# Exclude crash_record_id if it's in there, and columns already handled
numeric_cols = [col for col in numeric_cols if col not in ['crash_record_id', 'injuries_total', 
                                                              'hit_and_run_i', 'intersection_related_i', 
                                                              'private_property_i', 'work_zone_i', 'is_weekend']]

for col in numeric_cols:
    if df_clean[col].isna().sum() > 0:
        median_val = df_clean[col].median()
        print(f"{col}: {df_clean[col].isna().sum()} nulls -> filling with median = {median_val}")
        df_clean[col] = df_clean[col].fillna(median_val)

# Handle categorical columns with 'OTHER'
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
# Exclude crash_record_id (identifier)
categorical_cols = [col for col in categorical_cols if col != 'crash_record_id']

for col in categorical_cols:
    if df_clean[col].isna().sum() > 0:
        print(f"{col}: {df_clean[col].isna().sum()} nulls -> filling with 'OTHER'")
        df_clean[col] = df_clean[col].fillna('OTHER')

latitude: 286 nulls -> filling with median = 41.8623718176
longitude: 286 nulls -> filling with median = -87.6755386591
lat_bin: 286 nulls -> filling with median = 41.86
lng_bin: 286 nulls -> filling with median = -87.68


## Step 9: Cap Outliers

In [28]:
# Cap number of units involved at 10
df_clean.loc[df_clean['num_units'] > 10, 'num_units'] = 10

# Cap posted speed limit at 75 mph
df_clean.loc[df_clean['posted_speed_limit'] > 75, 'posted_speed_limit'] = 75

In [29]:
df_clean['hit_and_run_i'].value_counts()

hit_and_run_i
0    27368
1    14775
Name: count, dtype: int64

In [30]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42143 entries, 0 to 42143
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   crash_record_id         42143 non-null  object 
 1   beat_of_occurrence      42143 non-null  int64  
 2   crash_day_of_week       42143 non-null  int64  
 3   crash_hour              42143 non-null  int64  
 4   crash_type              42143 non-null  object 
 5   hit_and_run_i           42143 non-null  int64  
 6   num_units               42143 non-null  int64  
 7   injuries_total          42143 non-null  float64
 8   lighting_condition      42143 non-null  object 
 9   latitude                42143 non-null  float64
 10  longitude               42143 non-null  float64
 11  posted_speed_limit      42143 non-null  int64  
 12  road_defect             42143 non-null  object 
 13  roadway_surface_cond    42143 non-null  object 
 14  street_direction        42143 non-null  obj

In [33]:
df_clean['lighting_condition']

0        DARKNESS, LIGHTED ROAD
1                       UNKNOWN
2                      DARKNESS
3                      DARKNESS
4        DARKNESS, LIGHTED ROAD
                  ...          
42139    DARKNESS, LIGHTED ROAD
42140                      DAWN
42141                      DAWN
42142    DARKNESS, LIGHTED ROAD
42143                  DARKNESS
Name: lighting_condition, Length: 42143, dtype: object